In [20]:
import numpy as np, pandas as pd
import seaborn as sns, matplotlib.pyplot as plt
from langdetect import detect
from nltk.corpus import stopwords
from datetime import datetime, date, time

In [21]:
tw_df=pd.read_csv("/Users/rebeccaharrison/Documents/Python/Bootcamp/Hate_speech_project/livestreamfootball/twitter_stream.csv",index_col=0)
tw_df.head()


/var/folders/w3/j_v3f_rs4hlbh_rnzsc6j2rm0000gn/T/ipykernel_52294/1889024564.py:1: DtypeWarning: Columns (0,1,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  tw_df=pd.read_csv("/Users/rebeccaharrison/Documents/Python/Bootcamp/Hate_speech_project/livestreamfootball/twitter_stream.csv",index_col=0)


,id,created,text,possibly_sensitive,user,verified,rule
0,1.60165E+18,2022-12-10T18:29:58.000Z,@ecssxq Özlemişim🥹,FALSE,z' betül 🌪️,False,Players
1,1.60165E+18,2022-12-10T18:29:57.000Z,@Bilyoner 46' giroud,FALSE,ne yapayım hep aynı,False,Players
2,1.60165E+18,2022-12-10T18:29:58.000Z,RT @fifaworldcup_es: 💔🇵🇹 https://t.co/x38wBGPgar,FALSE,Nemanja,False,Worldcup
3,1.60165E+18,2022-12-10T18:29:58.000Z,Le Maroc en demi finale de la #WorldCup2022 ! ...,FALSE,TalkTime App by Cabriio !,False,England France
4,1.60165E+18,2022-12-10T18:29:57.000Z,quiero escuchar Angels fly https://t.co/FDiNpy...,FALSE,bleach,False,England France


In [22]:
red_df=pd.read_csv("/Users/rebeccaharrison/Documents/Python/Bootcamp/Hate_speech_project/livestreamfootball/reddit_stream.csv",index_col=0)
red_df['text']=red_df['title'].astype(str)+red_df['body'].astype(str)
red_df.drop(['id','url','score','title','body','num_comments'],axis=1, inplace=True)
red_df.head()
red_comm_df=pd.read_csv("/Users/rebeccaharrison/Documents/Python/Bootcamp/Hate_speech_project/livestreamfootball/reddit_comments_stream.csv",index_col=0)
red_comm_df=red_comm_df.rename(columns={'body':'text'})
red_comm_df.head()
red_comm_df = red_comm_df[red_comm_df["text"].str.contains("Hello! Thanks for your submission to /r/worldcup, your post is up and running!") == False]
red_comm_df.drop(['id','postid'],axis=1, inplace=True)
reddit_text=red_df.append(red_comm_df,ignore_index=True)
reddit_text.tail()

/var/folders/w3/j_v3f_rs4hlbh_rnzsc6j2rm0000gn/T/ipykernel_52294/709385851.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reddit_text=red_df.append(red_comm_df,ignore_index=True)


,subreddit,created,text
39288,NaN,1670214814.0,Japan vs Korea would be amazing
39289,NaN,1670235006.0,This is why Belgium and Germany got knocked out
39290,NaN,1670214868.0,Yes! It would be extremely fun and I’d be happ...
39291,NaN,1670198597.0,"At international level, FIFA has four official..."
39292,NaN,1670202669.0,yes. it was weird in switzerland vs serbia whe...


In [23]:
# Replace the string that proceeds the @ symbol with an empty string
tw_df['text'] = tw_df['text'].str.replace(r'@[^\s]+', '')
tw_df['text'] = tw_df['text'].str.replace(r'https[^\s]+', '')

# removing retweets
tw_df=tw_df[tw_df["text"].str.startswith("RT")==False]
tw_df

# removing empty tweets
tw_df=tw_df.dropna(subset=['text'])
# remove duplicates
tw_df=tw_df.drop_duplicates(subset=["text","user"])

tw_df



/var/folders/w3/j_v3f_rs4hlbh_rnzsc6j2rm0000gn/T/ipykernel_52294/3701934431.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  tw_df['text'] = tw_df['text'].str.replace(r'@[^\s]+', '')
/var/folders/w3/j_v3f_rs4hlbh_rnzsc6j2rm0000gn/T/ipykernel_52294/3701934431.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  tw_df['text'] = tw_df['text'].str.replace(r'https[^\s]+', '')


,id,created,text,possibly_sensitive,user,verified,rule
0,1.60165E+18,2022-12-10T18:29:58.000Z,Özlemişim🥹,FALSE,z' betül 🌪️,False,Players
1,1.60165E+18,2022-12-10T18:29:57.000Z,46' giroud,FALSE,ne yapayım hep aynı,False,Players
3,1.60165E+18,2022-12-10T18:29:58.000Z,Le Maroc en demi finale de la #WorldCup2022 ! ...,FALSE,TalkTime App by Cabriio !,False,England France
4,1.60165E+18,2022-12-10T18:29:57.000Z,quiero escuchar Angels fly,FALSE,bleach,False,England France
5,1.60165E+18,2022-12-10T18:29:58.000Z,WE LOVE LIAM PAYNE 💗\n #EnglandVsFrance #Engla...,FALSE,Judit,False,England France
...,...,...,...,...,...,...,...
499726,1601680000000000000.0,2022-12-10T21:01:23.000Z,Kane… I need my $300 back.,False,YOUNG CHOP’S RIGHT TITTY,False,Players
499728,1601680000000000000.0,2022-12-10T21:01:23.000Z,i’m wheezing because my boy is wearing his eng...,False,Papa G,False,England France
499729,1601680000000000000.0,2022-12-10T21:01:23.000Z,The only good thing about this is we get a Mor...,False,Andrea.,False,England France
499730,1601680000000000000.0,2022-12-10T21:01:24.000Z,So close Rashford. Well done bro. #ENG #FIFAWo...,False,Luenic Luke,False,Players


In [24]:
def detect_my(text):
   try:
       return detect(text)
   except:
       return 'unknown'

tw_df['language'] = tw_df['text'].apply(detect_my)

In [25]:


# removing stopwords from text

# Get a list of stopwords in English
stopwords = stopwords.words('english')

# Remove stopwords from the 'text' column in the dataframe
tw_df['text_nostop'] = tw_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))
tw_df.head()

,id,created,text,possibly_sensitive,user,verified,rule,language,text_nostop
0,1.60165E+18,2022-12-10T18:29:58.000Z,Özlemişim🥹,FALSE,z' betül 🌪️,False,Players,tr,Özlemişim🥹
1,1.60165E+18,2022-12-10T18:29:57.000Z,46' giroud,FALSE,ne yapayım hep aynı,False,Players,fr,46' giroud
3,1.60165E+18,2022-12-10T18:29:58.000Z,Le Maroc en demi finale de la #WorldCup2022 ! ...,FALSE,TalkTime App by Cabriio !,False,England France,fr,Le Maroc en demi finale de la #WorldCup2022 ! ...
4,1.60165E+18,2022-12-10T18:29:57.000Z,quiero escuchar Angels fly,FALSE,bleach,False,England France,ca,quiero escuchar Angels fly
5,1.60165E+18,2022-12-10T18:29:58.000Z,WE LOVE LIAM PAYNE 💗\n #EnglandVsFrance #Engla...,FALSE,Judit,False,England France,en,WE LOVE LIAM PAYNE 💗 #EnglandVsFrance #England...


In [26]:
tw_df['language'].value_counts()

en         175490
fr          38414
es          18155
pt          16664
unknown     12188
de           9531
id           5362
it           4962
tl           4797
ar           4464
so           4268
tr           3981
nl           3231
sw           3189
ca           3106
af           2871
no           2805
vi           2282
pl           2122
da           2045
ja           1982
et           1877
cy           1746
sv           1547
fi           1416
ro           1267
hu           1051
hr            909
sl            781
sq            680
lt            408
sk            383
cs            346
fa            255
lv            252
el            186
ur            154
th            110
ru             69
hi             63
ko             49
zh-cn          45
ta             28
uk             25
ml             22
ne             19
he             17
mk             13
bn             10
mr              8
bg              6
zh-tw           1
Name: language, dtype: int64

In [27]:
# removing non-english rows
tw_df = tw_df.loc[tw_df['language']=="en"]

tw_df

,id,created,text,possibly_sensitive,user,verified,rule,language,text_nostop
5,1.60165E+18,2022-12-10T18:29:58.000Z,WE LOVE LIAM PAYNE 💗\n #EnglandVsFrance #Engla...,FALSE,Judit,False,England France,en,WE LOVE LIAM PAYNE 💗 #EnglandVsFrance #England...
6,1.60165E+18,2022-12-10T18:29:57.000Z,What has Islam got to do with it?,FALSE,Antonio Plescia,False,Worldcup,en,What Islam got it?
11,1.60165E+18,2022-12-10T18:29:57.000Z,2002. That was also the Cup where Nigeria bo...,FALSE,Shiv Ramdas Traing To Rite Buk,True,England France,en,2002. That also Cup Nigeria bounced Argentina....
13,1.60165E+18,2022-12-10T18:29:58.000Z,Grant Wahl was a healthy individual just like...,FALSE,Porfirio Diaz,False,Worldcup,en,Grant Wahl healthy individual like many thousa...
18,1.60165E+18,2022-12-10T18:29:58.000Z,#ssfootball France 🇫🇷 2 England 🇬🇧 1,FALSE,Vinny Munda,False,England France,en,#ssfootball France 🇫🇷 2 England 🇬🇧 1
...,...,...,...,...,...,...,...,...,...
499724,1601680000000000000.0,2022-12-10T21:01:23.000Z,fuck it rematch croatia vs france,False,bryxn,False,England France,en,fuck rematch croatia vs france
499726,1601680000000000000.0,2022-12-10T21:01:23.000Z,Kane… I need my $300 back.,False,YOUNG CHOP’S RIGHT TITTY,False,Players,en,Kane… I need $300 back.
499728,1601680000000000000.0,2022-12-10T21:01:23.000Z,i’m wheezing because my boy is wearing his eng...,False,Papa G,False,England France,en,i’m wheezing boy wearing england tee siphakath...
499729,1601680000000000000.0,2022-12-10T21:01:23.000Z,The only good thing about this is we get a Mor...,False,Andrea.,False,England France,en,The good thing get Morocco - France semi


In [28]:
tw_df.to_csv('twitter_clean.csv')

In [29]:
# detecting languuge for reddit
reddit_text['language'] = reddit_text['text'].apply(detect_my)
# removing non english entries
reddit_text = reddit_text.loc[reddit_text['language']=="en"]

In [30]:
# removing stop words for reddit
reddit_text['text_nostop'] = reddit_text['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))
reddit_text.head()

,subreddit,created,text,language,text_nostop
0,worldcup,1670707019.0,Exciting! France advances to play against Croa...,en,"Exciting! France advances play Croatia, realit..."
1,worldcup,1670706978.0,Bellingham is good but nothing special. His na...,en,Bellingham good nothing special. His nationali...
2,worldcup,1670706906.0,Congratulations to the Budweiser Man of the Ma...,en,"Congratulations Budweiser Man Match, France’s ..."
3,worldcup,1670706846.0,Does google already know final game? Or just a...,en,Does google already know final game? Or typo? ...
4,worldcup,1670706802.0,We getting a rematch of 2018 or something diff...,en,We getting rematch 2018 something different?nan


In [31]:
reddit_text['language'].value_counts()

en    34015
Name: language, dtype: int64

In [32]:
# remove duplicates
reddit_text=reddit_text.drop_duplicates(subset=["text"])
#removing empty text
reddit_text=reddit_text.dropna(subset=['text'])

In [ ]:
reddit_text['date']=pd.to_datetime(reddit_text['created'],unit='s')
reddit_text=reddit_text.loc[reddit_text['date']>='2022-12-10']
reddit_text=reddit_text.loc[reddit_text['date'].dt.time >=time(18,29)]

In [33]:
# save to csv
reddit_text.to_csv('reddit_clean.csv')